In [ ]:
# mlp1.py
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Normalization, input
from tensorflow.keras import optimizers
import numpy as np; import sys

(AX, AY), (QX, QY) = mnist.load_data()
# inverte o fundo
AX=255-AX; QX=255-QX

# One hot encoding das labels
nclasses = 10
AY2 = keras.utils.to_categorical(AY, nclasses)
QY2 = keras.utils.to_categorical(QY, nclasses)

# normalização da entrada
nl, nc = AX.shape[1], AX.shape[2] #28, 28 [0, 255]

#1) Pseudo-normalizacao para intervalo [-0.5, +0.5]
#AX = (AX.astype('float32')/255.0)-0.5 # -0.5 a +0.5
#QX = (QX.astype('float32')/255.0)-0.5 # -0.5 a +0.5

#subtrai a media e divide pelo desvio padrão
#2) Normalizacao - distribuicao normal de media zero e desvio 1
#media=np.mean(AX); desvio=np.std(AX)
#AX=AX.astype('float32'); AX=AX-media; AX=AX/desvio; QX=QX.astype('float32'); QX=QX-media; QX=QX/desvio

#3) Normalizacao - inserir camada de normalizacao na rede
model = Sequential(); model.add(Input(shape=(nl,nc)))
model.add(Normalization()) #Normaliza
model.add(Flatten()) # transforma em um vetor porque rede densa não recebe matrizes
model.add(Dense(400, activation='sigmoid'))
model.add(Dense(nclasses, activation='sigmoid')) # camada de saida com numeros de neuronios igual de saida

# impressão de modelos
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='mlp1.png', show_shapes=True)
model.summary()

# optmizers adam
opt=optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])

# pega a camada com indice 0 e a de normalização pq a de INPUT não conta
# e faz a adaptação da entrada
model.get_layer(index=0).adapt(AX) #Calcula media e desvio
model.fit(AX, AY2, batch_size=100, epochs=40, verbose=2)
score = model.evaluate(QX, QY2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# salva o modelo treinado
model.save('mlp1.keras')

11490434/11490434 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 28, 28)            57        
 on)                                                             
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 400)               314000    
                                                                 
 dense_1 (Dense)             (None, 10)                4010      
                                                                 
Total params: 318067 (1.21 MB)
Trainable params: 318010 (1.21 MB)
Non-trainable params: 57 (232.00 Byte)
_________________________________________________________________
Epoch 1/40
600/600